<a href="https://colab.research.google.com/github/hirokimituya/stock-price-analysis/blob/main/technical_analysis/%E6%A0%AA%E4%BE%A1%E5%88%86%E6%9E%90%E9%96%A2%E6%95%B0_backtesting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [17]:
!pip install backtesting

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [15]:
# Ta-Libの代替としてpandas_taを利用している。Ta-Libの関数名を小文字にしたものがあるイメージ
!pip install pandas_ta

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.1/115.1 kB 3.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pandas_ta: filename=pandas_ta-0.3.14b0-py3-none-any.whl size=218921 sha256=6c8da46d0b9f87b6726d2af023783641b006672bb401031ee17afaf4aebb8c26
  Stored in directory: /root/.cache/pip/wheels/69/00/ac/f7fa862c34b0e2ef320175100c233377b4c558944f12474cf0
Successfully built pandas_ta


In [46]:
import pandas_datareader.data as pdr

# 株価データを取得するメソッド
def get_stock_data(code):
    """株価データを取得する

    :param code: 取得する株価データの銘柄コード
    :return: 株価データのデータフレーム
    """
    df = pdr.DataReader(f'{code}.JP', 'stooq').sort_index()
    return df

In [47]:
# バックテストの実行と結果の確認
from backtesting import Backtest
from backtesting import Strategy
from backtesting.lib import crossover
from backtesting.test import SMA
import datetime as dt

class SmaCross(Strategy):
    ns = 5   # 短期移動平均日数
    nl = 25   # 長期移動平均日数

    def init(self):
        # 短期移動平均
        self.smaS = self.I(SMA, self.data['Close'], self.ns)
        # 長期移動平均
        self.smaL = self.I(SMA, self.data['Close'], self.nl)
    
    def next(self):
        # smaS > smaL で買う
        if crossover(self.smaS, self.smaL):
            self.buy()
        # smaS < smaL で売る
        elif crossover(self.smaL, self.smaS):
            self.position.close()

def sma_cross_backtest(code, start, end=dt.date.today(), optimize=True, ns_periods=range(5, 75, 5), nl_periods=range(5, 75, 5), maximize='Return [%]'):
    """単純移動平均のゴールデンクロス／デッドクロスを用いた売買ルールのバックテストを実行する

    :param code: 取得する株価データの銘柄コード
    :param start: 取得する株価データの開始日
    :param end: 取得する株価データの終了日
    :param optimize: バックテストを最適化して実行するかどうか
    :param ns_periods: バックテストの最適化時に、短期移動平均線の値のリスト
    :param nl_periods: バックテストの最適化時に、長期移動平均線の値のリスト
    :param maximize: バックテストの最適化時に、どの値を最大にするように最適化するかの値（例: 'Return [%]' or 'SQN')
    """

    df = get_stock_data(code)
    data = df[start:end]

    bt = Backtest(data, SmaCross, trade_on_close=True)

    if optimize:
        # バックテストの最適化を実行
        result = bt.optimize(ns=ns_periods, nl=nl_periods, maximize=maximize, constraint=lambda p: p.ns < p.nl)
    else:
        # バックテスト実行
        result = bt.run()
    
    # 実行結果のデータを表示
    print(result)

    # 実行結果をグラフで表示
    bt.plot()

In [48]:
# RSIのゴールデンクロス／デッドクロスを用いた売買ルール
import pandas_ta as ta
import pandas as pd

def RSI(close, n1, n2):
    rsiS = ta.rsi(close, n1)
    rsiL = ta.rsi(close, n2)
    return rsiS, rsiL

class RSICross(Strategy):
    ns = 14 # 短期
    nl = 28 # 長期

    def init(self):
        self.rsiS, self.rsiL = self.I(RSI, pd.Series(self.data['Close']), self.ns, self.nl)
    
    def next(self):
        if crossover(self.rsiS, self.rsiL):
            self.buy()  # 短期 > 長期で買い
        elif crossover(self.rsiL, self.rsiS):
            self.position.close()   # 長期 > 短期で売り

def rsi_cross_backtest(code, start, end=dt.date.today(), optimize=True, ns_periods=range(5, 75, 5), nl_periods=range(5, 75, 5), maximize='Return [%]'):
    """RSIのゴールデンクロス／デッドクロスを用いた売買ルールのバックテストを実行する

    :param code: 取得する株価データの銘柄コード
    :param start: 取得する株価データの開始日
    :param end: 取得する株価データの終了日
    :param optimize: バックテストを最適化して実行するかどうか
    :param ns_periods: バックテストの最適化時に、短期移動平均線の値のリスト
    :param nl_periods: バックテストの最適化時に、長期移動平均線の値のリスト
    :param maximize: バックテストの最適化時に、どの値を最大にするように最適化するかの値（例: 'Return [%]' or 'SQN')
    """

    df = get_stock_data(code)
    data = df[start:end]

    bt = Backtest(data, RSICross, trade_on_close=True)

    if optimize:
        # バックテストの最適化を実行
        result = bt.optimize(ns=ns_periods, nl=nl_periods, maximize=maximize, constraint=lambda p: p.ns < p.nl)
    else:
        # バックテスト実行
        result = bt.run()
    
    # 実行結果のデータを表示
    print(result)

    # 実行結果をグラフで表示
    bt.plot()

In [49]:
# MACDのゴールデンクロス／デッドクロスを用いた売買ルール
def MACD(close, n1, n2, n3):
    macd_df = ta.macd(close, fast=n1, slow=n2, signal=n3)
    macd, macdsignal = macd_df.iloc[:, 0], macd_df.iloc[:, 2]
    return macd, macdsignal

class MACDCross(Strategy):
    n1 = 12
    n2 = 26
    n3 = 9

    def init(self):
        self.macd, self.macdsignal = self.I(MACD, pd.Series(self.data['Close']), self.n1, self.n2, self.n3)
    
    def next(self):
        if crossover(self.macd, self.macdsignal):
            self.buy()  # macd > シグナルで買い
        elif crossover(self.macdsignal, self.macd):
            self.position.close()   # シグナル > macdで売り

def macd_cross_backtest(code, start, end=dt.date.today(), optimize=True, n1_periods=range(5, 75, 5), n2_periods=range(10, 75, 5), n3_periods=range(10, 75, 5), maximize='Return [%]'):
    """MACDのゴールデンクロス／デッドクロスを用いた売買ルールのバックテストを実行する

    :param code: 取得する株価データの銘柄コード
    :param start: 取得する株価データの開始日
    :param end: 取得する株価データの終了日
    :param optimize: バックテストを最適化して実行するかどうか
    :param n1_periods: バックテストの最適化時に、短期移動平均線の値のリスト
    :param n2_periods: バックテストの最適化時に、長期移動平均線の値のリスト
    :param n2_periods: バックテストの最適化時に、シグナル生成時の移動平均線の値のリスト
    :param maximize: バックテストの最適化時に、どの値を最大にするように最適化するかの値（例: 'Return [%]' or 'SQN')
    """

    df = get_stock_data(code)
    data = df[start:end]

    bt = Backtest(data, MACDCross, trade_on_close=True)

    if optimize:
        # バックテストの最適化を実行
        result = bt.optimize(n1=n1_periods, n2=n2_periods, n3=n3_periods, maximize=maximize, constraint=lambda p: p.n1 < p.n2)
    else:
        # バックテスト実行
        result = bt.run()
    
    # 実行結果のデータを表示
    print(result)

    # 実行結果をグラフで表示
    bt.plot()

In [50]:
sma_cross_backtest(2395, '2021-1-1', '2022-3-31')

Backtest.optimize:   0%|          | 0/3 [00:00<?, ?it/s]

Start                     2021-01-04 00:00:00
End                       2022-03-31 00:00:00
Duration                    451 days 00:00:00
Exposure Time [%]                   58.223684
Equity Final [$]                    25879.916
Equity Peak [$]                     25879.916
Return [%]                          158.79916
Buy & Hold Return [%]              156.444719
Return (Ann.) [%]                  119.950259
Volatility (Ann.) [%]               86.710332
Sharpe Ratio                         1.383344
Sortino Ratio                        5.372377
Calmar Ratio                         4.901885
Max. Drawdown [%]                  -24.470233
Avg. Drawdown [%]                   -6.338194
Max. Drawdown Duration      101 days 00:00:00
Avg. Drawdown Duration       22 days 00:00:00
# Trades                                    3
Win Rate [%]                            100.0
Best Trade [%]                      71.243358
Worst Trade [%]                     24.324183
Avg. Trade [%]                    

In [51]:
rsi_cross_backtest(2395, '2021-1-1', '2022-3-31')

Backtest.optimize:   0%|          | 0/3 [00:00<?, ?it/s]

Start                     2021-01-04 00:00:00
End                       2022-03-31 00:00:00
Duration                    451 days 00:00:00
Exposure Time [%]                   50.328947
Equity Final [$]                    30505.886
Equity Peak [$]                     30505.886
Return [%]                          205.05886
Buy & Hold Return [%]              156.444719
Return (Ann.) [%]                  152.074286
Volatility (Ann.) [%]              101.482111
Sharpe Ratio                         1.498533
Sortino Ratio                        9.030565
Calmar Ratio                         8.573569
Max. Drawdown [%]                  -17.737571
Avg. Drawdown [%]                   -3.909969
Max. Drawdown Duration       94 days 00:00:00
Avg. Drawdown Duration       21 days 00:00:00
# Trades                                   23
Win Rate [%]                        47.826087
Best Trade [%]                       73.87128
Worst Trade [%]                     -4.215358
Avg. Trade [%]                    

In [52]:
macd_cross_backtest(2395, '2021-1-1', '2022-3-31')

/usr/local/lib/python3.10/dist-packages/backtesting/backtesting.py:1488: UserWarning: Searching for best of 1183 configurations.
  output = _optimize_grid()


Backtest.optimize:   0%|          | 0/4 [00:00<?, ?it/s]

Start                     2021-01-04 00:00:00
End                       2022-03-31 00:00:00
Duration                    451 days 00:00:00
Exposure Time [%]                   47.697368
Equity Final [$]                    29876.953
Equity Peak [$]                     30210.503
Return [%]                          198.76953
Buy & Hold Return [%]              156.444719
Return (Ann.) [%]                  147.758623
Volatility (Ann.) [%]              100.074438
Sharpe Ratio                         1.476487
Sortino Ratio                        8.568002
Calmar Ratio                         9.154101
Max. Drawdown [%]                  -16.141249
Avg. Drawdown [%]                    -4.20448
Max. Drawdown Duration       85 days 00:00:00
Avg. Drawdown Duration       21 days 00:00:00
# Trades                                   10
Win Rate [%]                             70.0
Best Trade [%]                      77.419673
Worst Trade [%]                    -10.353574
Avg. Trade [%]                    

# 分足データを取得する


In [53]:
!pip install yahoo_finance_api2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [56]:
from yahoo_finance_api2 import share as yapi2
import datetime as dt
from datetime import date,timedelta
import pandas as pd

# 株価データを取得するメソッド
def get_stock_data_yapi2(code):
    """株価データを取得する

    :param code: 取得する株価データの銘柄コード
    :return: 株価データのデータフレーム
    """
    data = yapi2.Share(f'{code}.T').get_historical(
        yapi2.PERIOD_TYPE_YEAR,
        10,
        yapi2.FREQUENCY_TYPE_DAY,
        1
    )

    df = pd.DataFrame(data)

    # タイムスタンプをDateTime型に変更
    df['datetime'] = pd.to_datetime(df['timestamp'], unit='ms')

    # 日本時間へ変換
    df['Date'] = df['datetime'] + dt.timedelta(hours=9)

    # インデックスをDateカラムに設定
    df.index = df['Date']

    # 不要なカラムを削除
    del df['Date'], df['datetime'], df['timestamp']

    # 各カラムの最初の文字を大文字に変換
    for col in df.columns:
        df[f'{col.capitalize()}'] = df[col]
        del df[col]

    return df

df = get_stock_data(4751)
df

,Open,High,Low,Close,Volume
Date,,,,,
2018-05-07,1454.17,1497.66,1444.51,1485.58,5.231481e+06
2018-05-08,1468.66,1531.48,1446.92,1502.47,5.851628e+06
2018-05-09,1512.15,1541.14,1492.81,1497.66,4.685443e+06
2018-05-10,1478.33,1483.16,1391.37,1417.94,9.530679e+06
2018-05-11,1434.85,1473.50,1432.43,1458.99,3.629376e+06
...,...,...,...,...,...
2023-04-25,1192.00,1203.00,1186.00,1192.00,3.534900e+06
2023-04-26,1180.00,1189.00,1168.00,1178.00,3.975300e+06
2023-04-27,1219.00,1230.00,1152.00,1153.00,1.342740e+07
